In [1]:
import yfinance as yf
import pandas as pd
import numpy as np






# Enter the ticker
#**Create list of stocks**
ticker = 'NINE'


# Moving Average Trigger Calculation

ma_stock_data = yf.Ticker(ticker).history(period='1y')
ma_50 = ma_stock_data['Close'].rolling(window=50).mean()
ma_200 = ma_stock_data['Close'].rolling(window=200).mean()
ma_results = []
for i in range(1, len(ma_stock_data)):
  
  if ma_50[i] > ma_200[i]:
    
    ma_results.append({'Date': ma_stock_data.index[i], 'MA_Trigger': 'buy'})
  
  elif ma_50[i] < ma_200[i]:
    
    ma_results.append({'Date': ma_stock_data.index[i], 'MA_Trigger': 'sell'})
ma_results_df = pd.DataFrame(ma_results)





# RSI Trigger Calculation
rsi_stock_data = yf.Ticker(ticker).history(period='14d')
rsi_stock_data['diff'] = rsi_stock_data['Close'].diff()
rsi_stock_data['pos_diff'] = np.where(rsi_stock_data['diff'] > 0, rsi_stock_data['diff'], 0)
rsi_stock_data['neg_diff'] = np.where(rsi_stock_data['diff'] < 0, abs(rsi_stock_data['diff']), 0)
rsi_stock_data['pos_ema'] = rsi_stock_data['pos_diff'].ewm(span=14).mean()
rsi_stock_data['neg_ema'] = rsi_stock_data['neg_diff'].ewm(span=14).mean()
rsi_stock_data['rs'] = rsi_stock_data['pos_ema'] / rsi_stock_data['neg_ema']
rsi_stock_data['rsi'] = 100 - (100 / (1 + rsi_stock_data['rs']))
rsi = rsi_stock_data['rsi']

rsi_results = []
for i in range(1, len(rsi_stock_data)):
  
  if rsi[i] < 30:
    
    rsi_results.append({'Date': rsi_stock_data.index[i], 'RSI_Trigger': 'buy'})
  
  elif rsi[i] > 70:
    
    rsi_results.append({'Date': rsi_stock_data.index[i], 'RSI_Trigger': 'sell'})
rsi_results_df = pd.DataFrame(rsi_results)



# Individual indicator results are grouped here
full_model_df = pd.concat([ma_results_df, rsi_results_df], axis=1, join='outer')
full_model_df.to_csv(r'/Users/zacharycraft/Desktop/Trading_Model/Current Version/RSI_MA.csv', index=False)







